In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


In TensorFlow 1, graphs were unavoidable (as were the complexities that came with
them): they were a central part of TensorFlow’s API. In TensorFlow 2, they are still 
there, but not as central, and much (much!) simpler to use. 

To demonstrate this, let’s
start with a trivial function that just computes the cube of its input:

In [2]:
def cube(x):
    return x ** 3

In [3]:
cube(2)

8

In [4]:
cube(tf.constant(2.0))

<tf.Tensor: id=2, shape=(), dtype=float32, numpy=8.0>

In [5]:
tf_cube = tf.function(cube)

In [6]:
tf_cube

In [7]:
tf_cube(2)

<tf.Tensor: id=10, shape=(), dtype=int32, numpy=8>

In [8]:
tf_cube(tf.constant(2))

<tf.Tensor: id=19, shape=(), dtype=int32, numpy=8>

**Under the hood, tf.function() analyzed the computations performed by the cube()
function and generated an equivalent computation graph!**

**Alternatively, we could have used
tf.function as a decorator**

In [12]:
@tf.function
def tf_cube(x):
    return x ** 3

The original Python function is still available via the TF Function’s python_function
attribute, in case you ever need it

In [13]:
tf_cube.python_function(2)

8

**When you write a custom loss function, a custom metric, a custom layer or
any other custom function, and you use it in a Keras model (as we did throughout
this chapter), Keras automatically converts your function into a TF Function, no need
to use tf.function()** . So most of the time, all this magic is 100% transparent.

You can tell Keras not to convert your Python functions to TF
Functions by setting dynamic=True when creating a custom layer
or a custom model. Alternatively, you can set run_eagerly=True
when calling the model’s compile() method.

TF Function generates a new graph for every unique set of input shapes and data
types, and it caches it for subsequent calls. For example, if you call tf_cube(tf.constant(10)) , a graph will be generated for int32 tensors of shape []. 
Then if you call tf_cube(tf.constant(20)) , the same graph will be reused. 
But if you then call
tf_cube(tf.constant([10, 20])) , a new graph will be generated for int32 tensors
of shape [2]. This is how TF Functions handle polymorphism (i.e., varying argument
types and shapes). However, **this is only true for tensor arguments:** 
if you pass numerical Python values to a TF Function, a new graph will be generated for every distinct
value: for example, calling tf_cube(10) and tf_cube(20) will generate two graphs.

**If you call a TF Function many times with different numerical
Python values, then many graphs will be generated, slowing down
your program and using up a lot of RAM. Python values should be
reserved for arguments that will have few unique values, such as
hyperparameters like the number of neurons per layer. This allows
TensorFlow to better optimize each variant of your model.**

To view the generated function’s source code, you can call ```tf.autograph.to_code(tf_cube.python_function)``` . The code is not
meant to be pretty, but it can sometimes help for debugging.

In [16]:
tf.autograph.to_code(tf_cube.python_function)

'def tf__tf_cube(x):\n  do_return = False\n  retval_ = ag__.UndefinedReturnValue()\n  do_return = True\n  retval_ = x ** 3\n  cond = ag__.is_undefined_return(retval_)\n\n  def get_state():\n    return ()\n\n  def set_state(_):\n    pass\n\n  def if_true():\n    retval_ = None\n    return retval_\n\n  def if_false():\n    return retval_\n  retval_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state)\n  return retval_\n'

If you call any external library, including NumPy or even the standard library,
this call will run only during tracing, it will not be part of the graph. Indeed, a
TensorFlow graph can only include TensorFlow constructs (tensors, operations,
variables, datasets, and so on). So make sure you use tf.reduce_sum() instead of
np.sum() , and tf.sort() instead of the built-in sorted() function, and so on
(unless you really want the code to run only during tracing).